In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from qiskit import QuantumCircuit, Aer, execute
from qiskit.providers.aer.noise import NoiseModel, depolarizing_error
from qiskit.visualization import plot_histogram

# Function to create a simple [[18, 12, 4]] code circuit
def create_code_circuit():
    qc = QuantumCircuit(18, 18)  # 17 qubits and 17 classical bits for intermediate measurements
    
    # Initialize logical |0> state (simplified example)
    for i in range(9):
        qc.h(i)
    
    # Encoding (simplified example)
    for i in range(9):
        qc.cx(i, i + 8)  # Adjust the indices to fit within 17 qubits
        qc.cx(i, (i + 9) % 18)
    
    qc.barrier()
    return qc

# Function to apply depolarizing error to a circuit
def add_depolarizing_error(qc, error_prob):
    noise_model = NoiseModel()
    single_qubit_error = depolarizing_error(error_prob, 1)
    two_qubit_error = depolarizing_error(error_prob, 2)
    
    noise_model.add_all_qubit_quantum_error(single_qubit_error, ['u3', 'u2', 'u1'])
    noise_model.add_all_qubit_quantum_error(two_qubit_error, ['cx'])
    
    return noise_model

# Function to measure and decode the circuit
def measure_and_decode(qc):
    qc.barrier()
    
    # Decoding (simplified example)
    for i in range(9):
        qc.cx(i, i + 8)  # Adjust the indices to fit within 17 qubits
        qc.cx(i, (i + 9) % 18)
    
    # Measure the qubits
    qc.measure(range(18), range(18))
    return qc

# Function to simulate the circuit and calculate error probability
def simulate_circuit(qc, error_prob, shots=1024):
    backend = Aer.get_backend('qasm_simulator')
    noise_model = add_depolarizing_error(qc, error_prob)
    qc_with_measurement = measure_and_decode(qc.copy())
    
    job = execute(qc_with_measurement, backend, noise_model=noise_model, shots=shots)
    result = job.result()
    counts = result.get_counts()
    
    # Calculate the total channel flip probability
    total_errors = sum(count for outcome, count in counts.items() if '1' in outcome[:9])
    total_counts = sum(counts.values())
    
    error_probability = total_errors / total_counts
    return error_probability

# Parameters
error_probabilities = np.linspace(0, 0.3, 31)  # Qubit error rates from 0 to 0.3
total_channel_flip_probs5 = []

# Create the quantum circuit for the [[18, 12, 4]] code
qc = create_code_circuit()

# Simulate for each error probability
for error_prob in error_probabilities:
    error_probability = simulate_circuit(qc, error_prob, shots=1024)
    total_channel_flip_probs5.append(error_probability)

# Plotting the results with TCFP on the x-axis
plt.figure(figsize=(8, 6))
plt.plot(total_channel_flip_probs5, error_probabilities, marker='o', linestyle='-', color='b')
plt.xlabel('Total Channel Flip Probability')
plt.ylabel('Qubit Error Rate')
plt.grid(True)
plt.show()
